In [13]:
from kafka import KafkaProducer
import time
import pandas as pd
dataDirectory = '../../data/'

In [14]:
producer = KafkaProducer(bootstrap_servers='kafka1:19092,kafka2:29092,kafka3:39092')

In [45]:
sensorDfs = []
sensors = [1,2,3,33,35 ]
for day in range(1,8,1):
    date = '2017-03-0{0}'.format(day)
    dfs = []
    for sensor in sensors:
        df = pd.read_csv(dataDirectory + date + '_sensor_{0}.csv'.format(sensor),
                         dtype={"measurement": float, "voltage": float})

        
        dfs.append(df)
    df = pd.concat(dfs)
    df.sort_values('time', inplace=True)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    df.index = df.index.time
    sensorDfs.append(df)

In [49]:
interval=10

#For synchronization with receiver (for the sake of the simulation), starts at a number of seconds multiple of 'interval'
current_time=time.time()
time_to_wait=interval-current_time%interval
time.sleep(time_to_wait)
for day in range(7):
    time_start=time.time()
    readings = sensorDfs[day].values.tolist()
    for r in readings:
        message=str(r)
        producer.send('ml',message.encode())
    time_to_send=time.time()-time_start
    print("Time to send "+str(len(readings))+" measurements (day "+str(day+1)+" ) : "+str(time_to_send))
    time.sleep(interval-time_to_send)

Time to send 9114 measurements (day 1 ) : 0.43648862838745117
Time to send 8514 measurements (day 2 ) : 0.4289400577545166
Time to send 7827 measurements (day 3 ) : 0.4199485778808594
Time to send 7967 measurements (day 4 ) : 0.3884150981903076
Time to send 8034 measurements (day 5 ) : 0.3910696506500244
Time to send 8099 measurements (day 6 ) : 0.40276646614074707
Time to send 8843 measurements (day 7 ) : 0.47673702239990234


In [19]:
message

"['2017-03-04', '1-0', 20.4294, 2.69964]"